In [ ]:
import numpy as np
import torch
import torchvision
import torch.nn as nn
import matplotlib.pyplot as plt
from torchvision import models
from torch.utils.data import DataLoader, TensorDataset, Dataset, SubsetRandomSampler
from sklearn.model_selection import KFold

In [ ]:
dataset = torch.load('/content/drive/.../data34_1.pt') # importing data set from your drive

In [ ]:
class gn(nn.Module):
    def __init__(self):
        super(gn, self).__init__()

        # bring resnet
        self.model = models.resnet18(pretrained= True) # recalling the resnet18 network
        self.model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False) # the images are 1 channel, so I have to change the input channel
        self.model.fc = nn.Linear(512 , 3, bias = True) # I have 3 classes here
    def forward(self, x):
        out = self.model(x)
        return out

my_resnet = gn()
num_epochs = 2



# loss
loss_fn = nn.CrossEntropyLoss()

# Optimizer
optimizer_fn = torch.optim.Adam(my_resnet.parameters(), lr=0.0001)

kf = KFold(n_splits=4, shuffle = True) # defining indices with shuffle=True allows you to extract data irregularly

for i in range(num_epochs):
    for (train_idx,test_idx) in kf.split(dataset):
      train_sub = SubsetRandomSampler(train_idx)
      test_sub = SubsetRandomSampler(test_idx)
      train_loader = DataLoader(dataset, batch_size=64, sampler=train_sub)
      test_loader = DataLoader(dataset, batch_size=64, sampler=test_sub)
      my_resnet.train()
      corrects = 0
      for j, (image, label) in enumerate(train_loader):
        out = my_resnet(image)
        predicted = torch.argmax(out, 1)
        corrects += torch.sum(predicted == label).item()
        loss_val = loss_fn(out, label)
        optimizer_fn.zero_grad() 
        loss_val.backward() 
        optimizer_fn.step() 
      acc_tr = 100*(corrects/len(train_idx))
      my_resnet.eval()
      corrects = 0
      for k, (imgs, lbls) in enumerate(test_loader):
        out = my_resnet(imgs)
        predicted = torch.argmax(out, 1)
        corrects += torch.sum(predicted == lbls).item()
      acc_te = 100*(corrects/len(test_idx))
      print ('epoch:',i+1,'| train loss:', round(loss_val.item(),3),', train accuracy:', round(acc_tr,3), '| validation accuracy:', round(acc_te,3))

epoch: 1 | train loss: 0.707 , train accuracy: 51.345 | validation accuracy: 53.993
epoch: 1 | train loss: 0.775 , train accuracy: 61.617 | validation accuracy: 51.606
epoch: 1 | train loss: 0.645 , train accuracy: 72.121 | validation accuracy: 73.22
epoch: 1 | train loss: 0.624 , train accuracy: 83.796 | validation accuracy: 78.472
epoch: 2 | train loss: 0.117 , train accuracy: 95.341 | validation accuracy: 95.009
epoch: 2 | train loss: 0.066 , train accuracy: 97.193 | validation accuracy: 97.179
epoch: 2 | train loss: 0.148 , train accuracy: 98.886 | validation accuracy: 99.436
